In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')

import tensorflow as tf
from tensorflow import keras

In [12]:
# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train_corrected.npy')

print(data_train.shape, labels_train.shape)

(270000, 5933) (5933,)


In [13]:
# Labels Encoding

labels_names = ['Nike',
                'Adidas',
                'Ford',
                'Honda',
                'General Mills',
                'Unilever',
                "McDonald's",
                'KFC',
                'Gators',
                '3M']

labels_names = ['Nike',
                'Adidas',
                'Ford',
                'Honda',
                'General_mills',
                'Unilever',
                "Mcdonalds",
                'KFC',
                'Gators',
                '3M']

In [14]:
#Resizing images

from PIL import Image
import numpy as np

image_resized = []
dim = (64, 64)

for i in range(len(data_train.T)):
    image = data_train[:, i].reshape((300, 300, 3))

    # Convert RGB to BGR
    bgr_image = image[..., ::-1]

    image_pil = Image.fromarray(bgr_image.astype('uint8'))  # Convert to PIL image
    resized_pil = image_pil.resize(dim, Image.ANTIALIAS)  # Resize using Pillow's resize function

    # Convert back to BGR
    resized_bgr = np.array(resized_pil)
    resized_rgb = resized_bgr[..., ::-1]

    image_resized.append(resized_rgb)

# Convert the list of resized images to a NumPy array
image_resized = np.array(image_resized)


/var/folders/87/nhh_mdjd5dj3s1fc5vmd_28r0000gn/T/ipykernel_82416/3583885791.py:16: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_pil = image_pil.resize(dim, Image.ANTIALIAS)  # Resize using Pillow's resize function


In [15]:
X_train_full = np.load('data_train.npy').T
t_train_full = np.load('labels_train_corrected.npy')

X_train_full.shape, t_train_full.shape# Loading Data
data_train = np.load('data_train.npy')
labels_train = np.load('labels_train_corrected.npy')

print(data_train.shape, labels_train.shape)

(270000, 5933) (5933,)


In [16]:
from sklearn.model_selection import train_test_split

# Training and Test sets
X_training, X_test, t_training, t_test = train_test_split(X_train_full, 
                                                  t_train_full, 
                                                  shuffle=True,
                                                  stratify=t_train_full,
                                                  test_size=0.15)
# Train and validation sets
X_train, X_val, t_train, t_val = train_test_split(X_training, 
                                                  t_training, 
                                                  shuffle=True,
                                                  stratify=t_training,
                                                  test_size=0.2)

X_training.shape, t_training.shape, X_train.shape, t_train.shape, X_val.shape, t_val.shape

((5043, 270000), (5043,), (4034, 270000), (4034,), (1009, 270000), (1009,))

In [17]:
X_training_reshaped = X_training.reshape(-1, 300, 300, 3)
X_test_reshaped = X_test.reshape(-1, 300, 300, 3)

# Reshape the input data to match the model's expected input shape
X_train_reshaped = X_train.reshape(-1, 300, 300, 3)
X_val_reshaped = X_val.reshape(-1, 300, 300, 3)

In [ ]:
base_model = keras.applications.resnet50.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
base_model.add()

In [ ]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

In [ ]:
# Freeze base model
base_model.trainable = False

In [ ]:
IMG_SIZE = 150

# .Input() instantiates a Keras tensor
inputs = keras.Input(shape=(300, 300, 3))
# Input layer

inputs_resized = tf.keras.layers.Resizing(IMG_SIZE, IMG_SIZE)(inputs)
# resizing input to match pretrained model

x = base_model(inputs_resized, training=False)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.

In [ ]:
inputs.shape, inputs_resized.shape, x.shape

In [ ]:
# Option 2: Flattening

# Convert features of shape `base_model.output_shape[1:]` to vectors
x_flatten = keras.layers.Flatten()(x)

x_flatten.shape

In [ ]:
# A Dense classifier with 10 units and softmax activation function
outputs = keras.layers.Dense(10, activation='softmax')(x_flatten)

model = keras.Model(inputs, outputs)

In [ ]:
type(model)

In [ ]:
inputs.shape, inputs_resized.shape, x.shape, outputs.shape

In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(learning_rate=0.01, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

model.fit(X_train_reshaped,t_train, epochs=5, batch_size=32,
          validation_data=(X_val_reshaped, t_val),
          callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])

# Again, in practice, you would run for a lot more epochs. 
# As well as perform the necessary hyperparameter tuning.

In [ ]:
# label predictions
y_test = np.argmax(model.predict(X_test_reshaped),axis=1)

y_test.shape

In [1]:
from sklearn.metrics import classification_report

print(classification_report(t_test, y_test, target_names=labels_names))

NameError: name 't_test' is not defined